In [6]:
print('Module 5_2: GNN_Virtual screening')

Module 5_2: GNN_Virtual screening


In [1]:
import torch
import pickle
import pandas as pd
import argparse
from torch_geometric.loader import DataLoader
from tqdm import tqdm

# Parse input

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-s','--screen_set', help='load the compound set to screen', required=True)
parser.add_argument('-f','--file_name', help="filename to save the result to", required=True)
args = parser.parse_args()

# Make functions

In [3]:
def score_model(model, data_loader):
    model.eval()
    scores = []
    smiles_list = []
    id_list = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Scoring"):
            batch = batch.to('cuda')
            out = model(batch)
            batch_scores = torch.exp(out)[:, 1].cpu().numpy()
            scores.extend(batch_scores)
            smiles_list.extend(batch.smiles) 
            id_list.extend(batch.cmpd_id)
    return smiles_list, id_list, scores


In [4]:
def save_sorted_scores(smiles_list, id_list, scores, output_csv_path):

    df = pd.DataFrame({
        'comp_id': id_list,
        'smiles': smiles_list,
        'GNN_prediction_score': scores,
    })

    df.to_csv(output_csv_path, index=False)
    print(f"Sorted scores saved to {output_csv_path}")



# Use functions

In [ ]:
model_path = 'gnn_model.pth'
model = torch.load(model_path, weights_only=False)
model.eval()
dataset_path = args.screen_set
with open(dataset_path, 'rb') as f:
    dataset = pickle.load(f)
data_loader = DataLoader(dataset, batch_size=512, num_workers=8)

smiles_list, id_list, scores = score_model(model, data_loader)

output_csv_path = args.file_name + '.csv'
save_sorted_scores(smiles_list, id_list, scores, output_csv_path)

/tmp/ipykernel_69995/4126684811.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)
Scoring: 100%|██████████| 8615/8615 [23:55<00:00,  6.00it/

Sorted scores saved to P09238_4m_VS_GNN.csv
